In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from functools import partial
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
from graspy.plot import heatmap
from graspy.simulations import er_np, sbm
from joblib import Parallel, delayed
from sklearn.metrics import adjusted_rand_score

from mgc.ksample import KSample

from src import generate_binary_sbms, estimate_embeddings

/Users/j1c/miniconda3/envs/graspy/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)


<IPython.core.display.Javascript object>

In [8]:
def run_experiment(m, block_1, block_2, p, delta, n_components, reps):
    omni_corrects = np.zeros((reps, block_1 + block_2))
    mase_corrects = np.zeros((reps, block_1 + block_2))

    for i in np.arange(reps).astype(int):
        pop1, pop2, true_labels = generate_binary_sbms(
            m=m, block_1=block_1, block_2=block_2, p=p, delta=delta
        )
        mase_corrects_tmp = []
        omni_corrects_tmp = []

        for method in ["mase", "omni"]:
            embeddings = estimate_embeddings(
                pop1, pop2, method, n_components, sample_space=True
            )
            for j in range(block_1 + block_2):
                test_stat, pval = KSample("Dcorr").test(
                    embeddings[:m, j, :], embeddings[m:, j, :]
                )

                if method == "mase":
                    mase_corrects[i, j] = pval
                else:
                    omni_corrects[i, j] = pval

    omni_powers = (omni_corrects <= 0.05).mean(axis=0)
    mase_powers = (mase_corrects <= 0.05).mean(axis=0)

    to_append = [m, p, delta, *omni_powers, *mase_powers]
    return to_append

<IPython.core.display.Javascript object>

In [9]:
spacing = 50

block_1 = 5  # different probability
block_2 = 15
p = 0.5
deltas = np.linspace(1 - p, 1, spacing + 1)
n_components = 2
reps = 25
ms = np.linspace(0, 250, spacing + 1)[1:].astype(int)

partial_func = partial(
    run_experiment,
    block_1=block_1,
    block_2=block_2,
    p=p,
    reps=reps,
    n_components=n_components,
)

args = [dict(m=m, delta=delta) for m, delta in product(ms, deltas)]
args = sum(zip(reversed(args), args), ())[: len(args)]

<IPython.core.display.Javascript object>

In [16]:
res = Parallel(n_jobs=-1, verbose=5)(delayed(partial_func)(**arg) for arg in args)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.8min finished


<IPython.core.display.Javascript object>

In [ ]:
cols = [
    "m",
    "p",
    "delta",
    *[f"omni_power_node={i}" for i in range(block_1 + block_2)],
    *[f"mase_power_node={i}" for i in range(block_1 + block_2)],
]
res_df = pd.DataFrame(res, columns=cols)
res_df.to_csv(
    f"./results/20200216_weighted_correct_nodes_{task_index}.csv", index=False
)